In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
ROOT_PATH = '/content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/';

In [ ]:
!pip install seqeval

In [ ]:
import pandas as pd 
import numpy as np
import json
from seqeval.metrics import classification_report

In [ ]:
def read_json(filename) :
  with open(filename, 'r', encoding="utf8") as f:
    obj = json.load(f)
  return obj

def write_json(obj, filename) :
  with open(filename, 'w', encoding="utf8") as outfile:
    json.dump(obj, outfile)
  print ("Successfully write JSON obj to", filename)

def write_tsv(obj, filename) :
  file = open(filename, "w", encoding="utf-8") 
  for tokens in obj :
    for token in tokens :
      file.write(token['token'] + "\t" + token['label'] + "\n")
    file.write("==\n\n")
  file.close()
  print ("Successfully write JSON obj to", filename)

def read_tsv(filename) :
  file = open(filename, "r") 
  obj = {}
  items = []
  words = file.read().split("==\n\n")
  sentences = [[tokens.split("\t") for tokens in word.split("\n")] for word in words]
  for tokens in sentences :
    item = []
    for token in tokens :
      if len(token) == 2 :
        tok_lab = {}
        tok_lab['token'] = token[0]
        tok_lab['label'] = token[1]
        item.append(tok_lab)
    if len(item) > 1 :
      items.append(item)
  obj['tokens_labels'] = items
  return obj

def write_to_txt(obj, filename, key=None) :
  file = open(filename, "w", encoding="utf-8") 
  for items in obj :
      texts = []
      for item in items :
        if key :
          texts.append(item[key])
        else :
          texts.append(item)
      file.write(" ".join(texts) + "\n")
  file.close()
  print ("Successfully write JSON obj to", filename)

def read_txt(filename) :
  results = open(filename, "r").readlines()
  return results

def make_unique(obj) :
  unique_list = []
  not_unique_list = 0

  for item in obj :
    if item in unique_list :
      not_unique_list += 1
    else :
      unique_list.append(item)

  obj = unique_list
  print ("Drop", not_unique_list, "Result", len(obj))
  return obj

def print_info(version, files) :
  for filename in files :
    print (filename)
    if version :
      path = ROOT_PATH + "dataset/version "+ str(version) + "/"
    else :
      path = ROOT_PATH + "dataset/"
    if "json" in filename :
      data = read_json(path + filename)
    elif "txt" in filename :
      data = read_tsv(path + filename)
    labels = [[object['label'] for object in objects] for objects in data["tokens_labels"]]
    labels_dummy = [["2-O" for object in objects] for objects in data["tokens_labels"]]
    print (classification_report(labels_dummy, labels))
    check_in_brands(data)

def check_in_brands(obj) :
  counts = [0, 0, 0, 0, 0, 0]
  brands = ["emina", "sariayu", "wardah", "inez", "pac", "viva"]
  result = {}
  list_of_token_label = []
  for tok_lab_s in obj['tokens_labels'] :
    for tok_lab in tok_lab_s :
      if tok_lab['token'].lower() in brands :
        list_of_token_label.append(tok_lab_s)
        counts[brands.index(tok_lab['token'].lower())] += 1
        break
  result['tokens_labels'] = list_of_token_label
  sum = 0
  for i in range(len(brands)) :
    sum += counts[i]
    print (brands[i], counts[i])
  print ("Not in brand", len(obj['tokens_labels']) - sum)
  print ("Sum", sum)

# Prepare for Train and Test Data

In [ ]:
obj = read_tsv(ROOT_PATH + "dataset/ecommerce/shopee-label.txt")
length = []
count = 0
for tokens in obj['tokens_labels'] :
  length.append(len(tokens))
  if (len(tokens) > 300):
    count += 1

print (max(length))
print (min(length))
count

261
25


0

In [ ]:
s = pd.Series(length)
s.value_counts()

40    29
32    13
63    12
48    12
53    10
      ..
78     1
77     1
74     1
73     1
25     1
Length: 74, dtype: int64

In [ ]:
obj = read_tsv(ROOT_PATH + "dataset/ecommerce/shopee-label.txt")
print (obj)
write_to_txt(obj['tokens_labels'], ROOT_PATH + "dataset/ecommerce/shopee.txt", "token")
write_json(obj, ROOT_PATH + "dataset/ecommerce/shopee-label.json")

{'tokens_labels': [[{'token': 'Wardah', 'label': '1-B-Merek'}, {'token': 'Everyday', 'label': '3-B-NamaProduk'}, {'token': 'Cheek', 'label': '3-I-NamaProduk'}, {'token': '&', 'label': '3-I-NamaProduk'}, {'token': 'Liptint', 'label': '3-I-NamaProduk'}, {'token': '01', 'label': '4-B-Varian'}, {'token': 'Red', 'label': '4-I-Varian'}, {'token': 'Set', 'label': '4-I-Varian'}, {'token': 'Glow', 'label': '4-I-Varian'}, {'token': '[SEP]', 'label': '2-O'}, {'token': 'Mengandung', 'label': '2-O'}, {'token': 'tujuh', 'label': '2-O'}, {'token': 'essence', 'label': '2-O'}, {'token': 'natural', 'label': '2-O'}, {'token': 'yang', 'label': '2-O'}, {'token': 'diklaim', 'label': '2-O'}, {'token': 'dapat', 'label': '2-O'}, {'token': 'memberi', 'label': '2-O'}, {'token': 'hidrasi', 'label': '2-O'}, {'token': 'pada', 'label': '2-O'}, {'token': 'bibir', 'label': '6-B-Penggunaan'}, {'token': '.', 'label': '2-O'}, {'token': 'Formulanya', 'label': '2-O'}, {'token': 'juga', 'label': '2-O'}, {'token': 'tidak', '

# Split Train and Test Set

In [ ]:
obj = read_tsv(ROOT_PATH + "dataset/ecommerce/shopee-label.txt")

brands = ["wardah"]

obj_result = {}
results = []
for tok_lab_s in obj['tokens_labels'] :
  for tok_lab in tok_lab_s :
    if tok_lab['token'].lower() in brands :
      results.append(tok_lab_s)
      break
obj_result['tokens_labels'] = results
print (len(obj_result['tokens_labels']))
write_tsv(obj_result['tokens_labels'], ROOT_PATH + "dataset/brand/shopee_" + brands[0] + "-label.txt")

147
Successfully write JSON obj to /content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/dataset/brand/shopee_wardah-label.txt


In [ ]:
# filenames = ["tokped_inez", "tokped_pac", "tokped_viva", "tokped_emina", "tokped_wardah", "tokped_sariayu", "shopee_emina", "shopee_wardah", "shopee_sariayu"]
# for filename in filenames :
#   obj = read_tsv(ROOT_PATH + "dataset/brand/" + filename + "-label.txt")
#   write_to_txt(obj['tokens_labels'], ROOT_PATH + "dataset/brand/" + filename + ".txt")
#   write_json(obj, ROOT_PATH + "dataset/brand/"+ filename + "-label.json")

In [ ]:
from random import seed, randint
seed(1)
indexes = {
    "shopee_emina" : {
      "sum" : 27,
      "train" : [],
      "test" : {
          1 : [],
          2 : [],
          3 : [],
      }
    },
    "shopee_sariayu" : {
      "sum" : 105,
      "train" : [],
      "test" : {
          1 : [],
          2 : [],
          3 : [],
      }
    },
    "shopee_wardah" : {
      "sum" : 153,
      "train" : [],
      "test" : {
          1 : [],
          2 : [],
          3 : [],
      }
    },
    "tokped_emina" : {
      "sum" : 390,
      "train" : [],
      "test" : {
          1 : [],
          2 : [],
          3 : [],
      }
    },
    "tokped_sariayu" : {
      "sum" : 370,
      "train" : [],
      "test" : {
          1 : [],
          2 : [],
          3 : [],
      }
    },
    "tokped_wardah" : {
      "sum" : 400,
      "train" : [],
      "test" : {
          1 : [],
          2 : [],
          3 : [],
      }
    },
    "tokped_inez" : {
      "sum" : 421,
      "train" : [],
      "test" : {
          1 : [],
          2 : [],
          3 : [],
      }
    },
    "tokped_pac" : {
      "sum" : 361,
      "train" : [],
      "test" : {
          1 : [],
          2 : [],
          3 : [],
      }
    },
    "tokped_viva" : {
      "sum" : 378,
      "train" : [],
      "test" : {
          1 : [],
          2 : [],
          3 : [],
      }
    }
}


In [ ]:
source_indexes = read_json(ROOT_PATH + "dataset/version final/index.json")
source_indexes.keys()

dict_keys(['shopee_emina', 'shopee_sariayu', 'shopee_wardah', 'tokped_emina', 'tokped_sariayu', 'tokped_wardah', 'tokped_inez', 'tokped_pac', 'tokped_viva'])

In [ ]:
for key in source_indexes.keys() :
  indexes[key]["train"] = source_indexes[key]["train"]
  print (key, len(indexes[key]["train"]))

shopee_emina 0
shopee_sariayu 0
shopee_wardah 0
tokped_emina 300
tokped_sariayu 0
tokped_wardah 300
tokped_inez 300
tokped_pac 300
tokped_viva 300


In [ ]:
# for key in indexes.keys() :
#   print (key)
#   if "sariayu" in key :
#     TRAIN_SIZE = 0
#     TEST_SIZE = 54
#   else :
#     if "tokped" in key :
#       TRAIN_SIZE = 300
#     else :
#       TRAIN_SIZE = 0

#     TEST_SIZE = 27
#   index_result = []
#   while len(index_result) < TRAIN_SIZE :
#     idx = randint(0, indexes[key]['sum'])
#     if idx not in index_result :
#       index_result.append(idx)
#   indexes[key]["train"] = index_result
#   print (len(index_result))

#   for i in range(3) :
#     index_result = []
#     while len(index_result) < TEST_SIZE :
#       idx = randint(0, indexes[key]['sum'])
#       if idx not in index_result and idx not in indexes[key]["train"] :
#         index_result.append(idx)
#     indexes[key]["test"][i+1] = index_result
#     print (len(index_result))

In [ ]:
# write_json(indexes, ROOT_PATH + "dataset/final version/index.json")

# Make Dataset

In [ ]:
indexes = read_json(ROOT_PATH + "dataset/version final/index.json")
indexes.keys()

dict_keys(['shopee_emina', 'shopee_sariayu', 'shopee_wardah', 'tokped_emina', 'tokped_sariayu', 'tokped_wardah', 'tokped_inez', 'tokped_pac', 'tokped_viva'])

In [ ]:
from random import seed, randint
seed(1)

In [ ]:
def create_dataset(keys, version, indexes) :
  texts = []
  texts_labels = []
  for key in keys :
    obj = read_tsv(ROOT_PATH + "dataset/brand/" + key + "-label.txt")
    sentences = read_txt(ROOT_PATH + "dataset/brand/" + key + ".txt")
    if version :
      for idx in indexes[key]["test"][str(version)] :
        try :
          texts_labels.append(obj["tokens_labels"][idx-1])
          texts.append(sentences[idx-1])
        except IndexError :
          print (key, idx-1, len(obj['tokens_labels']))
          while True :
            idx = randint(0, indexes[key]['sum'])
            if idx not in indexes[key]["test"][str(version)] :
              print ("Solved!", idx)
              texts_labels.append(obj["tokens_labels"][idx-1])
              texts.append(sentences[idx-1])
              break
    else :
      for idx in indexes[key]["train"] :
        texts_labels.append(obj["tokens_labels"][idx-1])
        texts.append(sentences[idx-1])
  print (len(texts))
  return texts, texts_labels

In [ ]:
test_name = 1
for i in range(3) :
  texts, texts_labels = create_dataset(["shopee_emina", "shopee_wardah"], i+1, indexes)
  write_tsv(texts_labels, ROOT_PATH + "dataset/version final/test " + str(i+1) + "/" + str(test_name) + "-label.txt")
  write_to_txt(texts, ROOT_PATH + "dataset/version final/test " + str(i+1) + "/" + str(test_name) + ".txt")

54
Successfully write JSON obj to /content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/dataset/version final/test 1/1-label.txt
Successfully write JSON obj to /content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/dataset/version final/test 1/1.txt
54
Successfully write JSON obj to /content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/dataset/version final/test 2/1-label.txt
Successfully write JSON obj to /content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/dataset/version final/test 2/1.txt
54
Successfully write JSON obj to /content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/dataset/version final/test 3/1-label.txt
Successfully write JSON obj to /content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/dataset/version final/test 3/1.txt


In [ ]:
train_keys = ["tokped_emina", "tokped_wardah", "tokped_inez", "tokped_pac", "tokped_viva"]

In [ ]:
train_texts = []
train_texts_labels = []
for key in train_keys :
  obj = read_tsv(ROOT_PATH + "dataset/brand/" + key + "-label.txt")
  texts = read_txt(ROOT_PATH + "dataset/brand/" + key + ".txt")
  for idx in indexes[key]["train"] :
    train_texts_labels.append(obj["tokens_labels"][idx-1])
    train_texts.append(texts[idx-1])
print (len(train_texts))
print ()

1500



In [ ]:
write_tsv(train_texts_labels, ROOT_PATH + "dataset/version final/train-label.txt")
write_to_txt(train_texts, ROOT_PATH + "dataset/version final/train.txt", None)

Successfully write JSON obj to /content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/dataset/version final/train-label.txt
Successfully write JSON obj to /content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/dataset/version final/train.txt


# Dataset Info

In [34]:
labels = ["Merek", "NamaProduk", "O", "Tekstur", "Penggunaan", "Varian", "Ukuran"]
def count_support(filename) :
  count = [0 for label in labels]
  obj = read_tsv(ROOT_PATH + filename)
  for tokens in obj["tokens_labels"] :
    for token in tokens :
      for i in range(len(labels)) :
        if labels[i] in token["label"] : # and "-B-" in token["label"] :
          count[i] += 1
          break
  for i in range(len(labels)) :
    print ("{0} {1}".format(count[i], labels[i]))

def create_vocabulary(filename, obj, train_voc) :
  vocabulary = {}
  for label in labels :
    vocs = []
    for tokens in obj["tokens_labels"] :
      for token in tokens :
        if label in token["label"] and token["token"] not in vocs:
          vocs.append(token["token"])
          break
    vocabulary[label] = vocs
  print (filename)
  for label in labels :
    count = 0
    for voc in vocabulary[label] :
      if voc not in train_voc[label] :
        count += 1
    print ("voc", label, len(vocabulary[label]), count)
  write_json(vocabulary, ROOT_PATH + "analyze/version final/" + filename + "-voc.json")

In [ ]:
files = ["shopee-label", "tokped_esw-label", "tokped_ipv-label"]
for file in files :
  print (file)
  count_support("dataset/ecommerce/" + file + ".txt")
  obj = read_tsv(ROOT_PATH + "dataset/ecommerce/" + file + ".txt")
  write_json(obj, ROOT_PATH + "dataset/ecommerce/" + file + ".json")
  print ()

shopee-label
393 Merek
481 NamaProduk
0 O
86 Tekstur
354 Penggunaan
451 Varian
250 Ukuran
Successfully write JSON obj to /content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/dataset/ecommerce/shopee-label.json

tokped_esw-label
2264 Merek
2075 NamaProduk
0 O
287 Tekstur
1458 Penggunaan
3430 Varian
593 Ukuran
Successfully write JSON obj to /content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/dataset/ecommerce/tokped_esw-label.json

tokped_ipv-label
3138 Merek
2877 NamaProduk
0 O
395 Tekstur
2174 Penggunaan
3371 Varian
337 Ukuran
Successfully write JSON obj to /content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/dataset/ecommerce/tokped_ipv-label.json



In [28]:
count_support("dataset/version final/train-label.txt")
obj = read_tsv(ROOT_PATH + "dataset/version final/train-label.txt")
create_vocabulary("train-label", obj)
write_json(obj, ROOT_PATH + "dataset/version final/train-label.json")

3636 Merek
13190 NamaProduk
98696 O
494 Tekstur
2714 Penggunaan
10169 Varian
1323 Ukuran
train-label
voc Merek 43
voc NamaProduk 499
voc O 1351
voc Tekstur 45
voc Penggunaan 50
voc Varian 624
voc Ukuran 131
Successfully write JSON obj to /content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/analyze/version final/train-label-voc.json
Successfully write JSON obj to /content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/dataset/version final/train-label.json


In [35]:
filenames = []
for j in range(4) :
  for i in range(3) :
    filenames.append("test " + str(i+1) + "/" + str(j+1) + "-label")

for filename in filenames :
  print (filename)
  count_support("dataset/version final/" + filename + ".txt")
  obj = read_tsv(ROOT_PATH + "dataset/version final/" + filename + ".txt")
  check_in_brands(obj)
  train_voc = read_json(ROOT_PATH + "analyze/version final/train-label-voc.json")
  create_vocabulary(filename, obj, train_voc)
  write_json(obj, ROOT_PATH + "dataset/version final/" + filename + ".json")
  print ()

test 1/1-label
63 Merek
454 NamaProduk
2701 O
27 Tekstur
79 Penggunaan
239 Varian
188 Ukuran
emina 27
sariayu 0
wardah 27
inez 0
pac 0
viva 0
Not in brand 0
Sum 54
test 1/1-label
voc Merek 2 0
voc NamaProduk 47 4
voc O 54 7
voc Tekstur 9 0
voc Penggunaan 8 1
voc Varian 43 6
voc Ukuran 24 12
Successfully write JSON obj to /content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/analyze/version final/test 1/1-label-voc.json
Successfully write JSON obj to /content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/dataset/version final/test 1/1-label.json

test 2/1-label
61 Merek
443 NamaProduk
2714 O
26 Tekstur
95 Penggunaan
232 Varian
182 Ukuran
emina 27
sariayu 0
wardah 27
inez 0
pac 0
viva 0
Not in brand 0
Sum 54
test 2/1-label
voc Merek 2 0
voc NamaProduk 48 3
voc O 54 8
voc Tekstur 9 0
voc Penggunaan 7 0
voc Varian 42 4
voc Ukuran 25 12
Successfully write JSON obj to /content/drive/My Drive/Tugas/Tugas Semest